Load the data

In [21]:
import pandas as pd

def load_local_file(file_path):
    local_df = pd.read_csv(file_path)
    return local_df 
    
finance_df = load_local_file('loan_payments.csv')   
print(finance_df.head())
print(finance_df.dtypes)
print(finance_df.info())

         id  member_id  loan_amount  funded_amount  funded_amount_inv  \
0  38676116   41461848         8000         8000.0             8000.0   
1  38656203   41440010        13200        13200.0            13200.0   
2  38656154   41439961        16000        16000.0            16000.0   
3  38656128   41439934        15000        15000.0            15000.0   
4  38656121   41439927        15000        15000.0            15000.0   

        term  int_rate  instalment grade sub_grade  ... recoveries  \
0  36 months      7.49      248.82     A        A4  ...        0.0   
1  36 months      6.99      407.52     A        A3  ...        0.0   
2  36 months      7.49      497.63     A        A4  ...        0.0   
3  36 months     14.31      514.93     C        C4  ...        0.0   
4  36 months      6.03      456.54     A        A1  ...        0.0   

  collection_recovery_fee  last_payment_date last_payment_amount  \
0                     0.0           Jan-2022              248.82   
1   

Transform Functions

In [22]:
class DataTransform:
    def __init__(self, dataframe) -> None:
        """
        initialises the class
        """
        self.dataframe = dataframe
        

    def remove_txt(self, column):
        """ 
        Removes superfluent text from column data
        """
        self.dataframe[column] = pd.to_numeric(self.dataframe[column].str.extract('(\d+)', expand=False))
        
        

    def correct_date_format(self):
       """
       Converts the date format of all coumns that contain a date
       """
       for column_name in self.dataframe.columns:
        if 'date' in column_name.lower():
            self.dataframe[column_name] = pd.to_datetime(self.dataframe[column_name])
    
    def correct_single_date_column(self, column_name):
       self.dataframe[column_name] = pd.to_datetime(self.dataframe[column_name])
            
    
    def type_to_category(self, column_name):
       """
       converts a column to a categorical data type
       """
       self.dataframe[column_name] = self.dataframe[column_name].astype('category')
       

mydata = DataTransform(finance_df)


In [43]:
finance_df['home_ownership'].unique()

['MORTGAGE', 'RENT', 'OWN', 'OTHER', 'NONE']
Categories (5, object): ['MORTGAGE', 'NONE', 'OTHER', 'OWN', 'RENT']

In [42]:
mydata.type_to_category('home_ownership')

In [36]:
"""
re-saves the updated DF to CSV
"""
finance_df.to_csv(r'C:\Users\Tom\EDS_Project_Loan_Finance\loan_payments_transformed.csv', index=False)

In [26]:
mydata.correct_date_format()


C:\Users\Tom\AppData\Local\Temp\ipykernel_15648\4113723130.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.dataframe[column_name] = pd.to_datetime(self.dataframe[column_name])
C:\Users\Tom\AppData\Local\Temp\ipykernel_15648\4113723130.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.dataframe[column_name] = pd.to_datetime(self.dataframe[column_name])
C:\Users\Tom\AppData\Local\Temp\ipykernel_15648\4113723130.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.dataframe[column_name] = pd.to_datetime(self.dataframe[column_name])
C:\Users\Tom\AppData\L

In [28]:
mydata.correct_single_date_column('earliest_credit_line')

C:\Users\Tom\AppData\Local\Temp\ipykernel_15648\4113723130.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.dataframe[column_name] = pd.to_datetime(self.dataframe[column_name])


In [31]:
mydata.remove_txt('term')

In [44]:
finance_df.dtypes

id                                      int64
member_id                               int64
loan_amount                             int64
funded_amount                         float64
funded_amount_inv                     float64
term                                  float64
int_rate                              float64
instalment                            float64
grade                                  object
sub_grade                              object
employment_length                    category
home_ownership                       category
annual_inc                            float64
verification_status                    object
issue_date                     datetime64[ns]
loan_status                            object
payment_plan                           object
purpose                              category
dti                                   float64
delinq_2yrs                             int64
earliest_credit_line           datetime64[ns]
inq_last_6mths                    

Create New Class and methods for DF Info

In [47]:
import pandas as pd

class DataFrameInfo:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def get_summary(self):
        """
        Returns a summary of the DataFrame including basic statistics.
        """
        return self.dataframe.describe()

    def get_missing_values(self):
        """
        Returns a DataFrame showing the count of missing values in each column.
        """
        return self.dataframe.isnull().sum().to_frame(name='missing_values')

    def get_data_types(self):
        """
        Returns a Series with data types of each column.
        """
        return self.dataframe.dtypes

    def get_unique_values(self, column_name):
        """
        Returns unique values in a specific column.
        """
        return self.dataframe[column_name].unique()

    def get_value_counts(self, column_name):
        """
        Returns value counts of unique values in a specific column.
        """
        return self.dataframe[column_name].value_counts()
    
    def get_shape_of_df(self):
        return self.dataframe.shape
    
    def count_distinct_values_categorical(self):
        """
        Returns the count of distinct values in categorical columns.
        Assumes categorical columns have 'category' dtype.
        """
        categorical_columns = self.dataframe.select_dtypes(include='category').columns
        distinct_values_count = {}
        for col in categorical_columns:
            distinct_values_count[col] = len(self.dataframe[col].cat.categories)
        return distinct_values_count

    def get_null_values_stats(self):
        """
        Generates a count and percentage of NULL values in each column.
        """
        null_count = self.dataframe.isnull().sum()
        null_percentage = (null_count / len(self.dataframe)) * 100
        null_stats = pd.DataFrame({
            'null_count': null_count,
            'null_percentage': null_percentage
        })
        return null_stats

df_info = DataFrameInfo(finance_df)

summary = df_info.get_summary()
print("DataFrame Summary:")
print(summary)

missing_values = df_info.get_missing_values()
print("\nMissing Values:")
print(missing_values)

data_types = df_info.get_data_types()
print("\nData Types:")
print(data_types)

unique_values_column = df_info.get_unique_values('employment_length')
print("\nUnique Values in Column:")
print(unique_values_column)

value_counts_column = df_info.get_value_counts('term')
print("\nValue Counts in Column:")
print(value_counts_column)

shape_of_df = df_info.get_shape_of_df()
print("\nShape of data is")
print(shape_of_df)

distinct_values_count = df_info.count_distinct_values_categorical()
print("\nDistinct Values Count in Categorical Columns:")
print(distinct_values_count)

# Generate count/percentage count of NULL values in each column
null_values_stats = df_info.get_null_values_stats()
print("\nNULL Values Stats:")
print(null_values_stats)


DataFrame Summary:
                 id     member_id   loan_amount  funded_amount  \
count  5.423100e+04  5.423100e+04  54231.000000   51224.000000   
mean   7.621797e+06  8.655350e+06  13333.076100   13229.509117   
min    5.552100e+04  7.069400e+04    500.000000     500.000000   
25%    7.594330e+05  9.587720e+05   7000.000000    7000.000000   
50%    7.084590e+06  8.709873e+06  12000.000000   12000.000000   
75%    8.860616e+06  1.052714e+07  18000.000000   18000.000000   
max    3.867612e+07  4.146185e+07  35000.000000   35000.000000   
std    9.571362e+06  1.031281e+07   8082.196709    8019.017599   

       funded_amount_inv          term      int_rate    instalment  \
count       54231.000000  49459.000000  49062.000000  54231.000000   
mean        12952.622979     42.606199     13.507328    400.013953   
min             0.000000     36.000000      5.420000     15.670000   
25%          6700.000000     36.000000     10.370000    224.205000   
50%         11300.000000     36.0000

In [53]:
columns_to_drop = ['mths_since_last_major_derog', 'mths_since_last_record', 'mths_since_last_delinq']
dropped_finance_df = finance_df.drop(columns_to_drop, axis=1)
print(dropped_finance_df.info())

KeyError: "['mths_since_last_delinq '] not found in axis"

In [ ]:
class Plotter:
    def __init__(self) -> None:
        pass

In [ ]:
class DataFramTransform:
    def __init__(self) -> None:
        pass